## Necessary packages:
```
langchain_huggingface
langchain
transformers
accelerate
bitsandbytes
ipywidget (for jupyter notebook)
langchain_experimental
tabulate

!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126 (for gpu inference, my cuda version is 12.6, so with suffix cu126)
```

Notice that we need to download the weights of the deepseek model from HuggingFace for further process. (in total about 20GB)

     

In [1]:
# check GPU availability;
# if not showing similar result as below, run the last line of the previous markdown.
import torch
print(torch.cuda.is_available(), torch.cuda.get_device_name(0))

True NVIDIA GeForce RTX 4060 Laptop GPU


In [2]:
# we don't need to change this block;
# this is the setting of the pipeline, which should hold same for the model
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline    # wrap the local model in HuggingFace chat wrapper

# transformers allows us to access checkpoints of pretrained models
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline    

model_id = "deepseek-ai/deepseek-coder-6.7b-instruct"                   # deepseek model which is good at coding
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code = True)

# nested quantization for memory efficiency
quantization = BitsAndBytesConfig(load_in_4bit = True, bnb_4bit_use_double_quant = True, bnb_4bit_compute_dtype = torch.float16)

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             torch_dtype = torch.float16,        # first load the weights as float16
                                             quantization_config = quantization, # then further quantize
                                             device_map = "auto",               # assign tasks to both GPU and CPU  
                                             trust_remote_code = True           # DeepSeek ships custom model class; must be allowed  
                                             )

pipe = pipeline(task = "text-generation",
                model=model,
                tokenizer = tokenizer,
                max_new_tokens = 1024,
                do_sample = False)

llm = HuggingFacePipeline(pipeline = pipe)
deepseek = ChatHuggingFace(llm=llm)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [3]:
# feed the data 
import pandas as pd
data = pd.read_excel("D:/syjnl/Documents/GenAI/AI project/genAI/Soccer.xlsx")

from langchain_experimental.agents import create_pandas_dataframe_agent
agent = create_pandas_dataframe_agent(deepseek, data, 
                                      verbose = True,                 # Check the intermediate thinking process
                                      allow_dangerous_code = True,    # must be allowed for code generation
                                      handle_parsing_errors = True
                                      )

print(agent.invoke("How many times 'Man United' appear in the column 'home'? Please give both the codes and the answer"))

D:\syjnl\anaconda3\envs\Myenv\Lib\site-packages\langchain_experimental\agents\agent_toolkits\pandas\base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(




> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Parsing LLM output produced both a final answer and a parse-able action:: <｜begin▁of▁sentence｜>You are an AI programming assistant, utilizing the Deepseek Coder model, developed by Deepseek Company, and you only answer questions related to computer science. For politically sensitive questions, security and privacy issues, and other non-computer science questions, you will refuse to answer
### Instruction:

You are working with a pandas dataframe in Python. The name of the dataframe is `df`.
You should use the tools below to answer the question posed of you:

python_repl_ast - A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.

Use th